# Ejercicio Práctico: Proyecto 
# ¿Están las estaciones meteorológicas de Colombia bien ubicadas para apoyar a sus principales municipios productores de cultivos?
 
**Programación SIG**   
2025-1
Avdoni Sánchez Reinoso

### Objetivo
* Evaluar la eficacia con la que la red nacional de estaciones meteorológicas cubre los municipios que concentran la mayor parte de la producción agrícola.

### Datos a usar  
* `municipios_colombia.shp` – límites municipales, EPSG 3116 
* `Evaluaciones_Agropecuarias_Municipales_EVA.csv` – totales de producción anual por municipio
* `Catalogo_Estaciones_IDEAM.csv` – latitud/longitud de las estaciones meteorológicas (convertir a EPSG 3116)

## Índice
* 1. **Cargar y armonizar**
* 2. **Enfoque en el año de cultivo**
* 3. **Productores del primer cuartil**
* 4. **Número de estaciones**
* 5. **Producción vs. cobertura**
* 6. **Comentario breve**

### * Librerías importadas que se usarán:

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
from pathlib import Path
from matplotlib.colors import Normalize

1. **Cargar y armonizar**  
* Lea los tres archivos.
* Complete los códigos municipales y asegúrese de que todas las geometrías cumplan con la norma EPSG 3116.

In [17]:
root_folder=r"/notebooks/datos/geodata_spatoper"
shp_path=root_folder+r"/municipios_colombia.shp"
df_muni = gpd.read_file(shp_path)
csv_path  = root_folder+"/Evaluaciones_Agropecuarias_Municipales_EVA.csv"
stations_csv_path=root_folder+"/Catalogo_Estaciones_IDEAM.csv"
muni = gpd.read_file(shp_path)
csv_eva = pd.read_csv(csv_path, encoding='utf-8')
csv_eva = csv_eva.rename(columns={'CÓD. MUN.': 'COD_MUNI'})
csv_eva['COD_MUNI'] = csv_eva['COD_MUNI'].astype(str).str.zfill(5)
muni['MPIO_CDPMP'] = muni['MPIO_CDPMP'].astype(str).str.zfill(5)
print("\nCRS:", muni.crs, "\n")              
print("\nVista de Muni_Shape:\n", muni.head())
print("\nVista de EVAs:\n", csv_eva.head())


CRS: EPSG:3116 


Vista de Muni_Shape:
   DPTO_CCDGO MPIO_CCDGO              MPIO_CNMBR MPIO_CDPMP  VERSION  \
0         18        001               FLORENCIA      18001     2018   
1         18        029                 ALBANIA      18029     2018   
2         18        094  BELÉN DE LOS ANDAQUÍES      18094     2018   
3         18        247             EL DONCELLO      18247     2018   
4         18        256               EL PAUJÍL      18256     2018   

           AREA   LATITUD   LONGITUD  STCTNENCUE  STP3_1_SI  ...  STP51_PRIM  \
0  2.547638e+09  1.749139 -75.558239     71877.0       32.0  ...     48848.0   
1  4.141221e+08  1.227865 -75.882327      2825.0       24.0  ...      1940.0   
2  1.191619e+09  1.500923 -75.875645      4243.0       54.0  ...      3541.0   
3  1.106076e+09  1.791386 -75.193944      8809.0        0.0  ...      7571.0   
4  1.234734e+09  1.617746 -75.234043      5795.0        0.0  ...      6072.0   

   STP51_SECU  STP51_SUPE  STP51_POST  STP51_13_E  

2. **Enfoque en el año de cultivo**
* Elija un cultivo (p. ej., arroz) y un año reciente.
* Calcule la producción municipal, la superficie sembrada y el rendimiento (t ha⁻¹).
    

In [41]:
## Se elije CEBOLLA DE BULBO
cebb_muni_2017 = csv_eva[(csv_eva["AÑO"] == 2017) & (csv_eva["CULTIVO"] == "CEBOLLA DE BULBO")].copy()
cbb_muni_res = cebb_muni_2017.groupby("COD_MUNI").agg({
    "MUNICIPIO": "first",
    "Producción\n(t)":"sum",
    "Área Sembrada\n(ha)": "sum",
    "Rendimiento\n(t/ha)": "mean"
}).reset_index()

##cbb_muni_res.head()
cebb_muni_2017.head()

,CÓD. \nDEP.,DEPARTAMENTO,COD_MUNI,MUNICIPIO,GRUPO \nDE CULTIVO,SUBGRUPO \nDE CULTIVO,CULTIVO,DESAGREGACIÓN REGIONAL Y/O SISTEMA PRODUCTIVO,AÑO,PERIODO,Área Sembrada\n(ha),Área Cosechada\n(ha),Producción\n(t),Rendimiento\n(t/ha),ESTADO FISICO PRODUCCION,NOMBRE \nCIENTIFICO,CICLO DE CULTIVO
57545,5,ANTIOQUIA,05001,MEDELLIN,HORTALIZAS,CEBOLLA,CEBOLLA DE BULBO,CEBOLLA DE BULBO,2017,2017A,10,9,126,14.0,HORTALIZA FRESCA,ALLIUM CEPA,TRANSITORIO
57546,15,BOYACA,15814,TOCA,HORTALIZAS,CEBOLLA,CEBOLLA DE BULBO,CEBOLLA DE BULBO,2017,2017A,500,450,15750,35.0,HORTALIZA FRESCA,ALLIUM CEPA,TRANSITORIO
57547,15,BOYACA,15238,DUITAMA,HORTALIZAS,CEBOLLA,CEBOLLA DE BULBO,CEBOLLA DE BULBO,2017,2017A,300,295,8260,28.0,HORTALIZA FRESCA,ALLIUM CEPA,TRANSITORIO
57548,15,BOYACA,15646,SAMACA,HORTALIZAS,CEBOLLA,CEBOLLA DE BULBO,CEBOLLA DE BULBO,2017,2017A,255,255,8925,35.0,HORTALIZA FRESCA,ALLIUM CEPA,TRANSITORIO
57549,15,BOYACA,15491,NOBSA,HORTALIZAS,CEBOLLA,CEBOLLA DE BULBO,CEBOLLA DE BULBO,2017,2017A,215,210,5040,24.0,HORTALIZA FRESCA,ALLIUM CEPA,TRANSITORIO


### Qué se hizo?:
1. Se filtra por año y cultivo del csv original para traer todos los datos de 2017 y de Cebolla de bulbo.
2. Se agrupa por código de municipio, y se trae los valores de "nombre de municipio" y para la producción y el área sembrada se suman los valores para que tenga en cuenta ambos periodos, pero el rendimiento se promedia, ya que este valor.
#### Nota
Sin embargo, el rendimiento se puede calcular con los nuevos datos y comparar que tan diferente queda respecto al promedidado.

In [42]:
cbb_muni_res ["New_rend_cal"] = cbb_muni_res["Producción\n(t)"]/cbb_muni_res["Área Sembrada\n(ha)"]
cbb_muni_res.head()

,COD_MUNI,MUNICIPIO,Producción\n(t),Área Sembrada\n(ha),Rendimiento\n(t/ha),New_rend_cal
0,05001,MEDELLIN,280,23,14.0,12.173913
1,15001,TUNJA,3920,196,20.0,20.000000
2,15180,CHISCAS,800,45,20.0,17.777778
3,15183,CHITA,1400,78,20.0,17.948718
4,15187,CHIVATA,203,12,19.0,16.916667


#### Nota:
* Se debe analizar el rendimiento recalculado, es decir, "New_rend_cal" ya que este dato se comporta como un promedio ponderado, y no uno simple.

3. **Productores del primer cuartil**
* Marcar los municipios en el cuartil superior de producción.
* Elaborar un mapa y una tabla (los 20 principales) de dichos municipios.

In [56]:
sort_prod_muni = cbb_muni_res[["COD_MUNI", "MUNICIPIO", "Producción\n(t)"]].sort_values("Producción\n(t)", ascending=False)
sort_prod_muni['prod_class'] = pd.qcut(sort_prod_muni["Producción\n(t)"], 4,
                             labels=['Q4','Q3','Q2','Q1'])
sort_prod_muni['prod_class'].value_counts()
##sort_prod_muni.head()

top_prod_muni = sort_prod_muni.head(20)
top_prod_muni

,COD_MUNI,MUNICIPIO,Producción\n(t),prod_class
36,15814,TOCA,24300,Q1
25,15646,SAMACA,17500,Q1
99,54670,SAN CALIXTO,16080,Q1
8,15238,DUITAMA,13720,Q1
34,15806,TIBASOSA,11016,Q1
18,15491,NOBSA,9984,Q1
95,54398,LA PLAYA,9680,Q1
69,25845,UNE,8036,Q1
96,54498,OCAÑA,7870,Q1
6,15224,CUCAITA,7720,Q1


Qué se hizo?:

1. Se ordenan los datos de producción de forma descendente **(Más alto al más bajo)** en un nuevo dataframe
2. Se realiza una clasificación por cuartiles.
3. A partir del df ordenado, se crea otro con los primeros 20 municipios.

4. **Número de estaciones**
* Convertir el archivo CSV de la estación a un GeoDataFrame.
* Unir espacialmente cada estación con su departamento (gpd.sjoin).
* Generar una tabla: número de estaciones por departamento.
* Crear:  
a) Un mapa de coropletas con el número de estaciones por departamento,  
b) Un gráfico de barras horizontales con el mismo número (en orden descendente).


In [ ]:
print(departs['num_estaciones'].dtype)

# Ver cuántos valores nulos hay en la columna 'DEPARTAMENTO'
print("Valores nulos en 'DEPARTAMENTO':", departs['DEPARTAMENTO'].isna().sum())

# Opcional: mostrar las filas donde 'DEPARTAMENTO' está nulo
print("\nFilas con 'DEPARTAMENTO' nulo:")
print(departs[departs['DEPARTAMENTO'].isna()])

5. **Producción vs. cobertura**
* Descomponer los municipios en departamentos (sumando la producción agrícola).
* Unir la tabla de recuento de estaciones a esta capa descompuesta.
* Generar un diagrama de dispersión de la producción departamental frente al recuento de estaciones; etiquetar los cuatro casos extremos (alta producción/baja cobertura, etc.).

6. **Comentario breve**
* Comente brevemente cualquier desajuste entre el volumen de producción y la cobertura de la estación; sugiera dos departamentos que podrían beneficiarse con estaciones adicionales.